In [4]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
data= pd.read_csv('DATASET_2019-10-24_light.txt') 
print(data.shape)


(535169, 407)


In [6]:
# #Filter the data

# #remove diabetic people 
# data=data[data.FIELD_16!=1]
# data=data[data.FIELD_23!=1]

# #remove hypertensive people who are diagnosed and on drugs
# # data=data[data.FIELD_15!=1]
# # data=data[data.FIELD_22!=1]

# #remove people who are diagnosed and on drugs for high lipids  
# # data=data[data.FIELD_17!=1]
# # data=data[data.FIELD_24!=1]

# print(data.shape)

In [7]:
newdata=data.copy()
print(newdata.shape)
newdata.head()

(535169, 407)


,Unnamed: 0,FIELD_1,FIELD_2,COMPARE,AGE,B_DAY,SEX,FIELD_3,FIELD_4,FIELD_5,...,S004424,S005200,S007400,S008501,S008502,S008503,S008504,S010101,S010102,CLASS
0,0,1000003,20130913,100000320130913,46.0,670212.0,0.0,NaN,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,14.0,0
1,1,1000003,20140613,100000320140613,47.0,670212.0,0.0,NaN,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,16.0,0
2,2,1000003,20150917,100000320150917,48.0,670212.0,0.0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,16.0,0
3,3,1000003,20160617,100000320160617,49.0,670212.0,0.0,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,18.0,0
4,4,1000003,20170608,100000320170608,50.0,670212.0,0.0,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,18.0,0


In [8]:
# for checking if there are two records per day
newdata_count_temp=newdata.groupby(by=['FIELD_1','FIELD_2']).size().reset_index(name='freq') 
# print(newdata_count_temp[newdata_count_temp.freq>1])

# group records by id and year to find out the number of tests per specific year
newdata_count_temp['Year']=(newdata_count_temp.FIELD_2 / 10000).astype(int)
newdata_count_peryear=newdata_count_temp.groupby(by=['FIELD_1','Year']).size().reset_index(name='freq')
# max(temp[temp.freq>1].freq)
newdata_count_peryear=newdata_count_peryear.sort_values(by='FIELD_1')
newdata_count_peryear.head()

# to get the number of years a patient visited
newdata_count_peryearG1= newdata_count_peryear.groupby(by=['FIELD_1']).size().reset_index(name='freq')
newdata_count_peryearG1.head()
print(newdata_count_peryearG1.freq.unique())
newdata_count_peryearG1=newdata_count_peryearG1[newdata_count_peryearG1.freq>1]
print(newdata_count_peryearG1.shape)
newdata_count_peryearG1.head()


[1 4 2 5 3 6]
(121704, 2)


,FIELD_1,freq
1,64,4
3,105,2
4,323,4
9,761,2
10,782,2


In [9]:
mylist=[]
for i in newdata_count_peryearG1.FIELD_1:
    vals=newdata_count_peryear[newdata_count_peryear.FIELD_1==i]
    mylist.append([i,np.sort(list(vals.Year))])

In [10]:
mylistPD=pd.DataFrame(mylist,columns=['FIELD_1','Years'])
print(mylistPD.shape)
mylistPD.tail(10)

(121704, 2)


,FIELD_1,Years
121694,5715973,"[2017, 2018]"
121695,5816083,"[2016, 2018]"
121696,5825127,"[2016, 2018]"
121697,5867953,"[2016, 2017]"
121698,5878782,"[2016, 2017]"
121699,5884342,"[2014, 2015]"
121700,5885504,"[2016, 2017]"
121701,5887822,"[2016, 2017]"
121702,5887825,"[2016, 2017]"
121703,5890185,"[2017, 2018]"


In [38]:
# newdata['Year']=(newdata.FIELD_2 / 10000).astype(int)

# x=[]
# target=[]
# counter=0

# for _id in mylistPD.FIELD_1:
#     ID=_id
#     years=mylistPD[mylistPD.FIELD_1==ID].Years.values
#     print(ID,years)

#     for y in range(len(years[0])-1):
#         dif=years[0][y+1] - years[0][y]
#         if(dif==1):
#             print(years[0][y])
#             counter=counter+1
#     print('====================>',counter)

In [11]:
newdata['Year']=(newdata.FIELD_2 / 10000).astype(int)

x=[]
target=[]

for _id in mylistPD.FIELD_1:
    ID=_id
    years=mylistPD[mylistPD.FIELD_1==ID].Years.values

    for y in range(len(years[0])-1):
        dif=years[0][y+1] - years[0][y]
        if(dif==1):
            
            tempdata=newdata.query('FIELD_1=='+str(ID)+' and Year=='+str(years[0][y])).reset_index()         
            x.append(tempdata.iloc[tempdata.shape[0]-1,:].values)

            tempdata=newdata.query('FIELD_1=='+str(ID)+' and Year=='+str(years[0][y+1])).reset_index()         
            target.append(tempdata.iloc[tempdata.shape[0]-1,:].values)



In [12]:
newdata.head()

,Unnamed: 0,FIELD_1,FIELD_2,COMPARE,AGE,B_DAY,SEX,FIELD_3,FIELD_4,FIELD_5,...,S005200,S007400,S008501,S008502,S008503,S008504,S010101,S010102,CLASS,Year
0,0,1000003,20130913,100000320130913,46.0,670212.0,0.0,NaN,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,18.0,14.0,0,2013
1,1,1000003,20140613,100000320140613,47.0,670212.0,0.0,NaN,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,16.0,0,2014
2,2,1000003,20150917,100000320150917,48.0,670212.0,0.0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,16.0,16.0,0,2015
3,3,1000003,20160617,100000320160617,49.0,670212.0,0.0,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16.0,18.0,0,2016
4,4,1000003,20170608,100000320170608,50.0,670212.0,0.0,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16.0,18.0,0,2017


In [13]:
# print(list(newdata.columns))

newcollist=newdata.columns
newcollist= np.insert(newcollist,0,'index')
newcollist

Index(['index', 'Unnamed: 0', 'FIELD_1', 'FIELD_2', 'COMPARE', 'AGE', 'B_DAY',
       'SEX', 'FIELD_3', 'FIELD_4',
       ...
       'S005200', 'S007400', 'S008501', 'S008502', 'S008503', 'S008504',
       'S010101', 'S010102', 'CLASS', 'Year'],
      dtype='object', length=409)

In [20]:
newdata.shape
tempcols=newcollist
# print(list(newdata.columns))

In [21]:
xlable=pd.DataFrame(x,columns=tempcols).drop(columns='Unnamed: 0')
xlable.tail(10)

,index,FIELD_1,FIELD_2,COMPARE,AGE,B_DAY,SEX,FIELD_3,FIELD_4,FIELD_5,...,S005200,S007400,S008501,S008502,S008503,S008504,S010101,S010102,CLASS,Year
193649,394319,5654226,20140616,565422620140616,31.0,830802.0,1.0,NaN,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,18.0,0,2014
193650,394320,5654226,20151125,565422620151125,32.0,830802.0,1.0,1,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,14.0,0,2015
193651,397432,5715973,20170313,571597320170313,43.0,740519.0,0.0,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16.0,12.0,0,2017
193652,405353,5867953,20160718,586795320160718,38.0,781211.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,12.0,0,2016
193653,405560,5878782,20161030,587878220161030,55.0,610524.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,17.0,0,2016
193654,405670,5884342,20141023,588434220141023,55.0,590727.0,1.0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,15.0,0,2014
193655,405696,5885504,20160923,588550420160923,34.0,820407.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.0,16.0,0,2016
193656,405745,5887822,20160831,588782220160831,44.0,721017.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,11.0,0,2016
193657,405747,5887825,20160930,588782520160930,44.0,721011.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,11.0,0,2016
193658,405790,5890185,20170724,589018520170724,45.0,720202.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,17.0,0,2017


In [22]:
targelable=pd.DataFrame(target,columns=tempcols).drop(columns='Unnamed: 0')
targelable.tail(10)

,index,FIELD_1,FIELD_2,COMPARE,AGE,B_DAY,SEX,FIELD_3,FIELD_4,FIELD_5,...,S005200,S007400,S008501,S008502,S008503,S008504,S010101,S010102,CLASS,Year
193649,394320,5654226,20151125,565422620151125,32.0,830802.0,1.0,1,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,14.0,0,2015
193650,394321,5654226,20160826,565422620160826,33.0,830802.0,1.0,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,15.0,0,2016
193651,397433,5715973,20180414,571597320180414,44.0,740519.0,0.0,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.0,16.0,0,2018
193652,405354,5867953,20170719,586795320170719,39.0,781211.0,0.0,NaN,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,13.0,0,2017
193653,405561,5878782,20170817,587878220170817,56.0,610524.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.0,15.0,0,2017
193654,405671,5884342,20151112,588434220151112,56.0,590727.0,1.0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,16.0,16.0,0,2015
193655,405697,5885504,20170703,588550420170703,35.0,820407.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19.0,18.0,0,2017
193656,405746,5887822,20170816,588782220170816,45.0,721017.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,11.0,0,2017
193657,405748,5887825,20170816,588782520170816,45.0,721011.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11.0,11.0,0,2017
193658,405791,5890185,20180524,589018520180524,46.0,720202.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,16.0,0,2018


In [23]:
print(xlable.shape)
print(targelable.shape)

(193659, 408)
(193659, 408)


In [24]:
# targelable.to_csv('_targelable4_withNa_AllColumns_OnDrugForLipid.txt',sep=',')
# xlable.to_csv('_xlable4_withNa_AllColumns_OnDrugForLipid.txt',sep=',')

In [25]:
targelable.to_csv('_TargetLable_forischemic.txt',sep=',')
xlable.to_csv('_XLable_forischemic.txt',sep=',')

In [26]:
print(xlable.shape)
print(targelable.shape)

(193659, 408)
(193659, 408)
